In [1]:
from brian2 import *
prefs.codegen.target = 'cython'  # 强制使用Cython
print("当前Brian2后端:", prefs.codegen.target)

# 测试一个最小网络
start_scope()
G = NeuronGroup(10, 'dv/dt = -v/(10*ms) : 1', threshold='v>10', reset='''v=0''')
S = Synapses(G, G, 'w : 1', on_pre='v += w')
S.connect(condition='i != j')  # 触发Cython编译
net = Network(G, S)
net.run(1*ms)
print("运行成功!")

当前Brian2后端: cython


INFO       No numerical integration method specified for group 'neurongroup', using method 'exact' (took 0.05s). [brian2.stateupdaters.base.method_choice]


运行成功!


In [1]:
import numpy as np
np.__config__.show()

{
  "Compilers": {
    "c": {
      "name": "msvc",
      "linker": "link",
      "version": "19.29.30156",
      "commands": "cl"
    },
    "cython": {
      "name": "cython",
      "linker": "cython",
      "version": "3.0.11",
      "commands": "cython"
    },
    "c++": {
      "name": "msvc",
      "linker": "link",
      "version": "19.29.30156",
      "commands": "cl"
    }
  },
  "Machine Information": {
    "host": {
      "cpu": "x86_64",
      "family": "x86_64",
      "endian": "little",
      "system": "windows"
    },
    "build": {
      "cpu": "x86_64",
      "family": "x86_64",
      "endian": "little",
      "system": "windows"
    }
  },
  "Build Dependencies": {
    "blas": {
      "name": "scipy-openblas",
      "found": true,
      "version": "0.3.27",
      "detection method": "pkgconfig",
      "include directory": "C:/Users/runneradmin/AppData/Local/Temp/cibw-run-j8vn_sl2/cp311-win_amd64/build/venv/Lib/site-packages/scipy_openblas64/include",
      "lib direct

f:\ljn\project1\.venv\Lib\site-packages\numpy\__config__.py:155: UserWarning: Install `pyyaml` for better output
  warnings.warn("Install `pyyaml` for better output", stacklevel=1)
